# svg_transforms

> Converting SVG transform directives to 2D matrix tranformations.

In [ ]:
#| default_exp svg_transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |hide

In [ ]:
#| export
from singleline_dataset.transforms import *


def _extract_g(svgroot):
    all_parsed = []
    all_g = svgroot.findall(".//{http://www.w3.org/2000/svg}g")
    for g in all_g:
        all_parsed.append(g.attrib)
    return all_parsed


def _get_transform_str(svgroot):
    all_g = _extract_g(svgroot)
    if not all_g:
        # print('no <g> tags found')
        return None
    elif len(all_g) > 1:
        return Exception(f"unexpected number of <g> tags found in SVG: {len(all_g)}")
    g = all_g[0]
    if "transform" not in g:
        # print('"transform" attribute not found in <g>')
        return None
    transform_str = g["transform"]
    return transform_str


def strip_tname(tname, item, expected_len):
    parts = item.replace(tname, "").replace("(", "").replace(")", "").split(",")
    nums = [float(n) for n in parts]
    if len(nums) != expected_len:
        raise Exception(
            f'unexpected length for transform instruction "{item}": {len(nums)} (expected {expected_len})'
        )
    return nums


def parse_transform_instruction(item):
    if item.startswith("translate"):
        tx, ty = strip_tname("translate", item, expected_len=2)
        # print(f"tx={tx}, ty={ty}")
        return translate_xform(tx, ty)
    elif item.startswith("rotate"):
        ang = strip_tname("rotate", item, expected_len=1)
        # print(f"ang={ang}")
        return rotate_xform(ang[0])
    elif item.startswith("scale"):
        sx, sy = strip_tname("scale", item, expected_len=2)
        # print(f"sx={sx}, sy={sy}")
        return scale_xform(sx, sy)
    else:
        raise Exception(f"runrecognized transform instruction: {item}")


def build_transforms(tstr):
    xforms = [parse_transform_instruction(part) for part in tstr.split(" ")]
    final = identity_xform()
    for x in reversed(xforms):
        final = final.dot(x)
    return final


def svg_to_transforms(svgroot):
    transform_str = _get_transform_str(svgroot)
    if not transform_str:
        return None
    return build_transforms(transform_str)

In [ ]:
#| export
def apply_path_attribute_transforms(paths, attributes):
    """
    Find and apply transform attributes specified inline on the <path> elements.
    (not used in final workflow, b/c the way I vectorized SVG's resulted in some skewed transform attributes)
    """
    all_coords = []
    for path, attrs in zip(paths, attributes):
        path_transform = identity_xform()
        path_transform_str = attrs.get("transform", None)
        if path_transform_str:
            print(f"path_transform_st={path_transform_str}")
            path_transform = build_transforms(path_transform_str)
        coords = apply_transform(coords, path_transform)
        all_coords.append(coords)
    return all_coords

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()